In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 💰 Nominal vs. Real GDP: Measuring What Matters

Gross Domestic Product (GDP) is a primary measure of a nation's economic output. However, simply looking at the total dollar value of goods and services produced (Nominal GDP) can be misleading because it's influenced by both changes in the *quantity* of goods produced and changes in their *prices*.

To understand the true change in economic output – the actual "stuff" being produced – we need to adjust for inflation. This gives us **Real GDP**.

This simulation visually demonstrates how Nominal GDP and Real GDP can diverge over time when there is inflation, even if the underlying growth in the money supply (which often drives nominal growth) is constant.

# 💵 Nominal GDP vs. 🧺 Real GDP

- **Nominal GDP:** Measures the value of goods and services produced using **current year prices**.
    - Formula (Conceptual): Nominal GDP = Current Prices × Current Quantities
    - It increases if either prices go up or quantities go up (or both).
    - Think of it as the total amount of "cash" changing hands for final goods and services.

- **Real GDP:** Measures the value of goods and services produced using **constant base-year prices**.
    - Formula (Conceptual): Real GDP = Base-Year Prices × Current Quantities
    - It increases *only* if the quantity of goods and services produced increases.
    - This measure isolates changes in actual *physical output* by removing the effect of price changes (inflation or deflation).
    - Think of it as the actual amount of "stuff" the economy produces.

**Why the difference matters:** We are usually more interested in Real GDP growth because it reflects changes in the standard of living and the economy's productive capacity. High nominal GDP growth driven purely by high inflation doesn't mean people are better off.

# 🧮 Simulation Logic & Calculation

This simulation uses a simplified approach:

1.  **Nominal GDP Growth:** We assume Nominal GDP grows at a constant rate equal to the specified `money_growth` rate. This is a simplification often related to the Quantity Theory of Money ($MV=PY$), where if velocity ($V$) and real output ($Y$) were constant, money supply growth ($M$) would directly translate to nominal GDP ($PY$) growth. In reality, the link is more complex.
    $$ \text{Nominal GDP}_t = \text{Nominal GDP}_0 \times (1 + \text{money\_growth})^t $$

2.  **Inflation:** We assume a constant `inflation` rate ($\pi$) over the period.

3.  **Real GDP Calculation:** Real GDP is calculated by deflating Nominal GDP using the assumed inflation rate. Essentially, we are adjusting the nominal value back to what it would be worth in "base year" (year 0 in this sim) dollars.
    $$ \text{Real GDP}_t = \frac{\text{Nominal GDP}_t}{(1 + \text{inflation})^t} $$

4.  **Implied Real Growth:** The simulation implicitly determines the real GDP growth rate ($g$) based on the difference between nominal growth and inflation:
    $$ (1 + g) \approx \frac{(1 + \text{money\_growth})}{(1 + \text{inflation})} $$
    So, $g \approx \text{money\_growth} - \text{inflation}$. The simulation calculates and displays this implied real growth rate.

The plot shows how the gap between Nominal GDP (the yellow line) and Real GDP (the green line) widens over time due to the cumulative effect of inflation. The shaded area represents the portion of nominal GDP growth attributable purely to price increases.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider # Use IntSlider for T
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def gdp_money_sim(nominal_growth=0.05, inflation=0.03, T=10, initial_gdp=1000):
    """
    Simulates and plots the divergence of Nominal and Real GDP over time
    given constant nominal growth and inflation rates.

    Args:
        nominal_growth (float): Assumed constant growth rate of Nominal GDP (e.g., 0.05 for 5%).
                                (Simplified proxy for money growth effects).
        inflation (float): Assumed constant inflation rate (e.g., 0.03 for 3%).
        T (int): Time Horizon in years.
        initial_gdp (float): Starting level of Nominal GDP (and Real GDP) at t=0.
    """
    # Ensure T is an integer
    T = int(T)
    if T < 1: T = 1 # Need at least 1 year

    # Create time arrays
    years_relative = np.arange(T) # Time steps 0, 1, ..., T-1
    years_actual = np.arange(2024, 2024 + T) # Example actual years

    # Calculate Nominal GDP path
    # Nominal_GDP(t) = Initial_GDP * (1 + nominal_growth)^t
    nominal_gdp = initial_gdp * (1 + nominal_growth)**years_relative

    # Calculate Real GDP path
    # Real_GDP(t) = Nominal_GDP(t) / (1 + inflation)^t
    # Or Real_GDP(t) = Initial_GDP * [(1 + nominal_growth) / (1 + inflation)]^t
    real_gdp = nominal_gdp / (1 + inflation)**years_relative

    # Calculate the implied constant real growth rate
    # (1 + real_growth) = (1 + nominal_growth) / (1 + inflation)
    if abs(1 + inflation) < 1e-9: # Avoid division by zero if inflation is -100%
        real_growth = -1.0 if nominal_growth > -1 else 0 # Or handle as appropriate
    else:
        real_growth = (1 + nominal_growth) / (1 + inflation) - 1

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot Nominal GDP
    ax.plot(years_actual, nominal_gdp, label=f'Nominal GDP (g={nominal_growth:.1%})', linewidth=2.5, color='gold', marker='o', markersize=4)

    # Plot Real GDP
    ax.plot(years_actual, real_gdp, label=f'Real GDP (g≈{real_growth:.1%})', linewidth=2.5, color='seagreen', marker='s', markersize=4)

    # Fill the gap representing inflation effect
    ax.fill_between(years_actual, real_gdp, nominal_gdp, color='lightgray', alpha=0.5, label='Inflation Effect')

    # Add annotations for final values
    ax.text(years_actual[-1], nominal_gdp[-1], f' ${nominal_gdp[-1]:,.0f}', color='darkgoldenrod', va='center', ha='left', fontsize=9)
    ax.text(years_actual[-1], real_gdp[-1], f' ${real_gdp[-1]:,.0f}', color='darkgreen', va='center', ha='left', fontsize=9)


    ax.set_title("💵 Nominal vs. Real GDP Over Time 🧺")
    ax.set_xlabel("Year")
    ax.set_ylabel(f"GDP (Base Year = {years_actual[0]})")
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend()
    # Format y-axis as currency/plain number
    try:
        # Use FuncFormatter for better control if needed, basic formatting here
        ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    except Exception:
        pass # Ignore formatting errors if they occur

    plt.tight_layout()
    plt.show()

    # --- Display Growth Rates ---
    results_md = f"""
    ### 📈 Growth Rate Summary:

    * **Assumed Nominal GDP Growth:** {nominal_growth:.2%} per year
    * **Assumed Inflation Rate:** {inflation:.2%} per year
    * **Implied Real GDP Growth:** {real_growth:.2%} per year
        * *(Approximation: Nominal Growth - Inflation = {nominal_growth - inflation:.2%})*
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'} # Allow longer descriptions
interact(gdp_money_sim,
         nominal_growth=FloatSlider(value=0.05, min=-0.05, max=0.15, step=0.005, description='Nominal Growth Rate:', readout_format='.1%', style=style),
         inflation=FloatSlider(value=0.03, min=-0.05, max=0.15, step=0.005, description='Inflation Rate:', readout_format='.1%', style=style),
         T=IntSlider(value=10, min=3, max=40, step=1, description='Years (T):', style=style),
         initial_gdp=FloatSlider(value=1000, min=100, max=10000, step=100, description='Initial GDP ($):', style=style, readout_format=',.0f')
        );

interactive(children=(FloatSlider(value=0.05, description='Nominal Growth Rate:', max=0.15, min=-0.05, readout…

# 🔍 Key Insight

The simulation clearly shows that **Nominal GDP growth is not the same as real economic growth**.

> Nominal GDP reflects both changes in output and changes in prices.
> Real GDP isolates the change in actual output, providing a better measure of economic progress and changes in the availability of goods and services.

When analyzing economic data or news reports, always be mindful of whether figures are reported in nominal or real terms, especially during periods of significant inflation or deflation.

# 💰 Cash Money and GDP

More dollars **doesn’t always mean** more goods.

When we look at GDP:
- **Nominal GDP** = Prices × Quantities
- **Real GDP** = **Quantities**, adjusted for inflation



# 🛒 Metaphor:
If you go shopping with $50 and buy:
- 10 items at $5 each → real GDP = 10

Now if prices rise to $10:
- With $100, you still only get 10 items → real GDP unchanged

But nominal GDP **doubled**.



# 🔍 Key Insight:
> **Nominal GDP growth ≠ real growth**  
> We strip out inflation to understand **what’s really changing**.

# 📝 Guided Student Exercise: Nominal vs. Real GDP
Apply your understanding!

1. **Suppose Nominal GDP grows at 8% per year and inflation is 5%.**
    - What is the implied real GDP growth rate?
    - Use the interactive plot above to check your answer over a 10-year period.

2. **Experiment:**
    - Set inflation higher than nominal GDP growth. What happens to real GDP?
    - Try negative inflation (deflation). How does this affect the gap between nominal and real GDP?

3. **Challenge:**
    - If initial GDP is $2,000, nominal growth is 4%, and inflation is 2%, what is real GDP after 5 years?

---
# 🌍 Real-World Data Extension: U.S. Nominal and Real GDP
Let's see how this works in the real world. We'll plot U.S. Nominal and Real GDP using FRED data.

```python
import pandas as pd
import plotly.graph_objects as go
nom_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=GDP'
real_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?id=GDPC1'
nom = pd.read_csv(nom_url)
real = pd.read_csv(real_url)
nom['DATE'] = pd.to_datetime(nom['DATE'])
real['DATE'] = pd.to_datetime(real['DATE'])
fig = go.Figure()
fig.add_trace(go.Scatter(x=nom['DATE'], y=nom['GDP'], name='Nominal GDP ($ Billions)', yaxis='y1'))
fig.add_trace(go.Scatter(x=real['DATE'], y=real['GDPC1'], name='Real GDP (2012$ Billions)', yaxis='y2'))
fig.update_layout(title='U.S. Nominal vs. Real GDP', xaxis_title='Year', yaxis=dict(title='Nominal GDP', side='left'), yaxis2=dict(title='Real GDP', overlaying='y', side='right'))
fig.show()
```

- What do you notice about the gap between nominal and real GDP over time?
- How do recessions and inflationary periods show up in the data?

---
# 📚 Further Reading & Resources
- [Mankiw, N. G. (2021). *Macroeconomics* (11th Edition), Chapter 2: Measuring the Economy.](https://www.macmillanlearning.com/college/us/product/Macroeconomics/p/1319243584)
- [FRED: Nominal GDP (GDP)](https://fred.stlouisfed.org/series/GDP)
- [FRED: Real GDP (GDPC1)](https://fred.stlouisfed.org/series/GDPC1)
- [Marginal Revolution University: GDP and Price Indices](https://mru.org/courses/principles-economics-macroeconomics/gdp-and-price-indices)

---
# 🎨 Tips for Visual Exploration
- Use the interactive plot to try different growth and inflation rates.
- Discuss with classmates: Why is real GDP a better measure of living standards than nominal GDP?
- How might the model change if velocity or output growth is not constant?

---
# 🚀 Next Steps
- Continue to the next notebook to explore inflation, money, and the Quantity Theory in more depth!